# In Selector Group Chat 'description' plays a major role 

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
import asyncio

from codecs import StreamReader
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console

api_key = os.getenv("OPEN_ROUTER_API_KEY") 

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="openai/gpt-4o-mini",    
    model_info={
        "family": "openai",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

In [18]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [19]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [20]:
web_search_agent = AssistantAgent(
    name="WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

In [21]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [22]:
data_analyst_agent = AssistantAgent(
    name="DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

# Termination Condition


In [23]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [24]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

# Selector Group Chat has the model client but in the RoundRobinGroupChat -> no need (since it goes in the sequential way no need of brain to it)

In [25]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [27]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [28]:
await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To accomplish this task, I will break it into two main subtasks. 

1. Find the Miami Heat player with the highest points in the 2006-2007 season.
2. Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.

Now, I will assign these tasks.

1. WebSearchAgent : Find out which Miami Heat player had the highest points in the 2006-2007 season.
2. WebSearchAgent : Gather information on the total rebounds for this player for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for the identified player. 

Once these tasks are complete, I will summarize the findings.
---------- 

TaskResult(messages=[TextMessage(id='35aea4ef-3040-4868-8e65-479583ba4e0a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 13, 22, 54, 85436, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='d13dc5fe-b057-443b-9cfe-382d0bb232e7', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=179), metadata={}, created_at=datetime.datetime(2026, 2, 15, 13, 23, 2, 331932, tzinfo=datetime.timezone.utc), content="To accomplish this task, I will break it into two main subtasks. \n\n1. Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nNow, I will assign these tasks.\n\n1. WebSearch

# Custom Selector function

In [30]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import List, Sequence
def my_slector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_slector_fun
)

task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [31]:
await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
I will follow the same steps to re-evaluate the task.

1. Find the Miami Heat player with the highest points in the 2006-2007 season.
2. Gather information on the total rebounds for this player for the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.

Now, I will assign these tasks.

1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent : Gather information on the total rebounds for this player (Dwayne Wade) for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for Dwayne Wade. 


TaskResult(messages=[TextMessage(id='428200c5-b991-4120-9384-ea49a5bc5145', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 14, 3, 23, 508440, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='efacd205-23db-4fbe-8ca0-b74518d3c067', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=636, completion_tokens=206), metadata={}, created_at=datetime.datetime(2026, 2, 15, 14, 3, 28, 512625, tzinfo=datetime.timezone.utc), content='I will follow the same steps to re-evaluate the task.\n\n1. Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. Gather information on the total rebounds for this player for the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentage change in total rebounds between the 2007-20

In [32]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import List, Sequence
def my_slector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):
    if messages[-1].source == web_search_agent.name:
        return data_analyst_agent.name
    return None

selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_slector_fun
)

task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [33]:
await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
I will follow the same steps as before to ensure that I provide accurate and concise information.

1. Find the Miami Heat player with the highest points in the 2006-2007 season.
2. Gather information on the total rebounds for this player for the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.

Now, I will assign these tasks.

1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent : Gather information on the total rebounds for this player (Dwyane Wade) for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2

TaskResult(messages=[TextMessage(id='85b83f72-e015-42b3-aa28-1af3d035d005', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 15, 34, 59, 267341, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c665f9e3-fb93-4067-9a21-9aa10c039101', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=1317, completion_tokens=212), metadata={}, created_at=datetime.datetime(2026, 2, 15, 15, 35, 5, 207403, tzinfo=datetime.timezone.utc), content='I will follow the same steps as before to ensure that I provide accurate and concise information.\n\n1. Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. Gather information on the total rebounds for this player for the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentag

# Custom Candidate Function

In [ ]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

# User Proxy Agent

In [40]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=5)
combined_termination = text_mention_termination | max_message_termination

def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await selector_team.reset()
selector_team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (PlanningAgent) ----------
To address your request, I will break it down into manageable subtasks:

1. WebSearchAgent : Search for the Miami Heat player with the highest points scored during the 2006-2007 season.
2. WebSearchAgent : Search for the total rebounds of the identified player for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.

Please proceed with the tasks.
---------- TextMessage (UserProxyAgent) ----------
DISAPPROVE
---------- TextMessage (PlanningAgent) ----------
I understand your concern. Let's revise the plan to ensure we gather the required information more effectively. 

1. WebSearchAgent : Search for the Miami Heat player with the highest points scored during the 2006-2007 season.
2. WebSearchAgent : Search for the total rebounds of that player in the 2007-2008 season.
3. WebSearchAgent : Search for the total rebounds of that player in th

TaskResult(messages=[TextMessage(id='2d59aec0-be37-466e-8816-825ad8c49b6c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 16, 19, 27, 455781, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='972a9b83-9928-4919-8354-ba1298ac04a0', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=111), metadata={}, created_at=datetime.datetime(2026, 2, 15, 16, 19, 30, 728474, tzinfo=datetime.timezone.utc), content='To address your request, I will break it down into manageable subtasks:\n\n1. WebSearchAgent : Search for the Miami Heat player with the highest points scored during the 2006-2007 season.\n2. WebSearchAgent : Search for the total rebounds of the identified player for the 2007-2008 and 2008-2009 seasons.\n3. 